In [1]:
import wrds

In [2]:
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
conn.list_libraries().sort()
type(conn.list_libraries())

list

In [4]:
sp500_tickers = []
with open('sp500.txt', 'r') as f:
    for line in f:
        sp500_tickers.append(line.strip())
sp500_tickers[:5]

['AAPL', 'NVDA', 'MSFT', 'AMZN', 'GOOGL']

In [5]:
from tqdm import tqdm

In [14]:
ibes_tickers = []
for t in tqdm(sp500_tickers):
    try:
        data = conn.raw_sql(f"""SELECT ticker, oftic, cname, estimator, analys, FPI, MEASURE, VALUE, FPEDATS, ANNDATS, ANNTIMS, ACTUAL, ANNDATS_ACT, ANNTIMS_ACT
                         FROM tr_ibes.det_epsus
                         WHERE oftic = '{t}'
                         and anndats >= '01/01/2024'
                         and fpi = '6'
                         """)
        if data.shape[0] == 0:
            print(f"No data for {t}")
        else:
            ibes_ticker = data['ticker'].iloc[0]
            ibes_tickers.append((t, ibes_ticker))
        # print(t, data.shape)
    except Exception as e:
        print(f"Error for {t}: {e}")

 76%|███████▌  | 382/501 [00:19<00:05, 22.63it/s]

No data for VRSN


 79%|███████▊  | 394/501 [00:19<00:04, 21.80it/s]

No data for L


 82%|████████▏ | 412/501 [00:20<00:04, 21.78it/s]

No data for NWS


100%|██████████| 501/501 [00:24<00:00, 20.43it/s]

No data for AMTM


In [17]:
for oftic, ibes in tqdm(ibes_tickers):
    data = conn.raw_sql(f"""SELECT ticker, oftic, cname, estimator, analys, FPI, MEASURE, VALUE, FPEDATS, ANNDATS, ANNTIMS, ACTUAL, ANNDATS_ACT, ANNTIMS_ACT
                         FROM tr_ibes.det_epsus
                         WHERE ticker = '{ibes}'
                         and anndats >= '01/01/2020'
                         and fpi = '6'
                         """)
    data.to_csv(f"data/ibes/{oftic}.csv", index=False)

100%|██████████| 497/497 [00:29<00:00, 16.78it/s]


In [54]:
ibes_data = conn.raw_sql("""SELECT ticker, oftic, cname, estimator, analys, FPI, MEASURE, VALUE, FPEDATS, ANNDATS, ANNTIMS, ACTUAL, ANNDATS_ACT, ANNTIMS_ACT
                         FROM tr_ibes.det_epsus
                         WHERE oftic = 'AAPL'
                         and anndats >= '01/01/2024'
                         and fpi = '6'
                         """)
ibes_data

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

In [21]:
ibes_data

,ticker,oftic,cname,estimator,analys,fpi,measure,value,fpedats,anndats,anntims,actual,anndats_act,anntims_act
0,AAPL,AAPL,APPLE,282.0,108607.0,6,EPS,2.350,2024-12-31,2024-10-31,20:19:00,NaN,None,None
1,AAPL,AAPL,APPLE,79.0,115953.0,6,EPS,2.020,2023-12-31,2024-01-02,16:37:00,2.18,2024-02-01,16:30:00
2,AAPL,AAPL,APPLE,3037.0,49552.0,6,EPS,2.092,2023-12-31,2024-01-02,01:00:00,2.18,2024-02-01,16:30:00
3,AAPL,AAPL,APPLE,873.0,153096.0,6,EPS,2.160,2023-12-31,2024-01-08,06:01:00,2.18,2024-02-01,16:30:00
4,AAPL,AAPL,APPLE,183.0,125669.0,6,EPS,2.100,2023-12-31,2024-01-18,06:07:00,2.18,2024-02-01,16:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,AAPL,AAPL,APPLE,29.0,58377.0,6,EPS,2.370,2024-12-31,2024-11-01,00:23:00,NaN,None,None
113,AAPL,AAPL,APPLE,4280.0,49595.0,6,EPS,2.360,2024-12-31,2024-11-01,06:26:00,NaN,None,None
114,AAPL,AAPL,APPLE,3225.0,195817.0,6,EPS,2.370,2024-12-31,2024-11-03,22:52:00,NaN,None,None
115,AAPL,AAPL,APPLE,4201.0,119509.0,6,EPS,2.300,2024-12-31,2024-11-04,05:50:00,NaN,None,None
